# GradientTape
自动微分器主要的功能是用来计算微分的值的，通常用于梯度下降法。\
除此之外，自动微分器可以用来追踪tensor的训练过程。\
其工作原理是在进程中执行微分操作，而后记录在tape里面，关闭了以后我们需要的值记录在tape里面，拿出来看就行了。

In [22]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf


## scalar的梯度计算

In [8]:
#------------------一元梯度----------------------
print("1D derivation")
x = tf.constant(value = 3.0)   #微分器里面必须是浮点数！！！
with tf.GradientTape(persistent=True,watch_accessed_variables = True) as tape:
    #persistent表示可以反复使用。 
    tape.watch(x)  #watch确定记录的变量
    
    
    #定义function
    y1=x*2
    y2=x*x+2
    y3=x*x+2*x
    
    #求导
    dy1_dx = tape.gradient(target=y1,sources=x)#自变量是x，函数是y
    dy2_dx = tape.gradient(target=y2,sources=x)#自变量是x，函数是y
    dy3_dx = tape.gradient(target=y3,sources=x)#自变量是x，函数是y
    
   
    #输出
    print("dy1_dx:",dy1_dx)
    print("dy2_dx:",dy2_dx)
    print("dy3_dx:",dy3_dx)
    
    
#------------------二元梯度-----------------------
print("2D derivation")
x = tf.constant(value=3.0)
y = tf.constant(value=2.0)

with tf.GradientTape(persistent=True) as tape:
    tape.watch([x,y])
    
    #构造函数
    z=x*x*y+x*y
    
    #求导
    dz_dx=tape.gradient(sources=x,target=z)
    dz_dx_dy = tape.gradient(sources=[x,y],target=z)
    
    print(dz_dx)
    print(dz_dx_dy)

W0902 16:22:52.472019 4734318016 backprop.py:968] Calling GradientTape.gradient on a persistent tape inside its context is significantly less efficient than calling it outside the context (it causes the gradient ops to be recorded on the tape, leading to increased CPU and memory usage). Only call GradientTape.gradient inside the context if you actually want to trace the gradient in order to compute higher order derivatives.
W0902 16:22:52.473538 4734318016 backprop.py:968] Calling GradientTape.gradient on a persistent tape inside its context is significantly less efficient than calling it outside the context (it causes the gradient ops to be recorded on the tape, leading to increased CPU and memory usage). Only call GradientTape.gradient inside the context if you actually want to trace the gradient in order to compute higher order derivatives.
W0902 16:22:52.475494 4734318016 backprop.py:968] Calling GradientTape.gradient on a persistent tape inside its context is significantly less ef

1D derivation
dy1_dx: tf.Tensor(2.0, shape=(), dtype=float32)
dy2_dx: tf.Tensor(6.0, shape=(), dtype=float32)
dy3_dx: tf.Tensor(8.0, shape=(), dtype=float32)
2D derivation
tf.Tensor(14.0, shape=(), dtype=float32)
[<tf.Tensor: id=184, shape=(), dtype=float32, numpy=14.0>, <tf.Tensor: id=185, shape=(), dtype=float32, numpy=12.0>]


## 复杂函数的梯度求解
其求梯度的对象不但可以是直接的函数，还可以是复合函数，矩阵函数，神经网络中复杂的函数之间的复合导数，就可以用这个方法求导。

In [29]:
import numpy as np
#创建一个矩阵
x = tf.ones((2,2))

#在微分器里面建立需要求导的自变量和因变量
#由于GradientTape是一个进程，因此开启进程需要记得关闭，微分器的所有操作都要
#在with... as..里面进行。操作过后，python会自动关闭进程。
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)#reduce_sum是按行列，或者整个矩阵中每个元素都加起来的和
    z = tf.multiply(y,y)
    
    # 这里我们建立了两个函数
    # y=f(x)
    # z=f(y)
    # 因此z也是x的复合函数！dz_dy = t.gradient(z, y)

在上面， GradientTape中默认的call的次数是1，就是说使用过这个tape以后，这个tape就被抹去了，因此下面的内容不能call两次。

In [21]:
#查看一个tape里面记录的内容。
dz_dx = t.gradient(z,x)

#看看对不对。
for i in [0,1]:
    for j in [0,1]:
        assert dz_dx[i][j].numpy() == 8 
        # 理论上导数矩阵所有元素都应该都是8，因此，我们判定，如果不是8的话，
        # 这里assert就起到一个判定并中断的作用，如果表达式是false，
        # 那么就强制产生error。
        

中间变量也可以被tape记录下来。\
我们watch的x，但是当我调用z对y的导数的时候，y的也被记录下来了。

In [30]:
dz_dy = t.gradient(z, y)
print(dz_dy.numpy())

8.0


### persistant
每次调用，都会自动释放tape的内存，我们可以选择调用之后不释放，即使用persistant method。

In [37]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = x*x
    z = y*y

dz_dx = t.gradient(z,x)
dy_dx = t.gradient(y,x)

#释放内存，用完了就销毁tape。
del t

## 控制微分流
在微分器打开的时候，也是可以通过python的控制语言打断并改变其微分流的。\
这可以实现更复杂的微分。

In [43]:
def f(x, y):
    output = 1.0
    for i in range(y):
        if i > 1 and i < 5:
            output = tf.multiply(output, x)
    return output

def grad(x, y):
    with tf.GradientTape() as t:
        t.watch(x)
        out = f(x, y)    # 可以看到f(x,y)的值并不是一个固定的函数。
                         # f(x,y)具体是什么函数还取决于x或者y的值
                         # 可即便是这样，我们仍然可以对这样不确定的函数求导。
                         # 这一切的实现，都是由于即使在GradientTape打开的时候，仍然可以操控数据流。
    return t.gradient(out, x)

x = tf.convert_to_tensor(2.0)


#检验一下答案，如果有问题就产生error。

assert grad(x, 6).numpy() == 12.0
assert grad(x, 5).numpy() == 12.0
assert grad(x, 4).numpy() == 4.0

## 高阶导数
由于GradientTape接受嵌套，因此高阶导数可以利用tape的嵌套来实现。

In [48]:
x = tf.Variable(1.0)

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        y = x*x*x
    
    dy_dx = t2.gradient(y,x)
d2y_dx2 = t.gradient(dy_dx,x)

assert dy_dx.numpy() == 3.0
assert d2y_dx2.numpy() == 6.0

print(dy_dx.numpy(),d2y_dx2.numpy())


3.0 6.0
